In [37]:
import numpy as np
from scipy.io import wavfile
from scipy import signal
from pydub import AudioSegment

In [38]:
path = "./Bontempi-B3-C5.wav"
asegment = AudioSegment.from_wav(path)
asegment = asegment.set_channels(1)
asegment.export("mono.wav",format="wav")

s,a = wavfile.read("mono.wav")

In [39]:
#Сократим длинный сигнал
a = a[:len(a)//10]

In [40]:
N = len(a)
Fs = s
M = 26
f1 = 0
fh = Fs

In [41]:
x = np.ones(N)
X = np.zeros([N,2])

In [42]:
for i in range(N):
    Sc = 0
    Ss = 0
    for j in range(N):
        Sc += x[j]*np.cos(2*np.pi*i*j/N)
        Ss += -x[j]*np.sin(2*np.pi*i*j/N)
    X[i,0] = Sc
    X[i,1] = Ss
    

In [43]:
def B(x):
    return 1125*np.log(1+x/700)

def rB(x):
    return 700*(np.exp(x/1125)-1)

In [44]:
def f(m,M,f1,fh,Fs,N):
    return N/Fs*rB(B(f1)+m*(B(fh)-B(f1))/(M+1))

In [45]:
def H(k,m,f1,fh,M,N,Fs):
    if k < f(m,M,f1,fh,Fs,N):
        return 0
    elif f(m,M,f1,fh,Fs,N) <= k < f(m+1,M,f1,fh,Fs,N):
        return (k-f(m,M,f1,fh,Fs,N))/(f(m+1,M,f1,fh,Fs,N)-f(m,M,f1,fh,Fs,N))
    elif f(m+1,M,f1,fh,Fs,N) <= k <= f(m+2,M,f1,fh,Fs,N):
        return (f(m+2,M,f1,fh,Fs,N)-k)/(f(m+2,M,f1,fh,Fs,N)-f(m+1,M,f1,fh,Fs,N))
    else:
        return 0


In [46]:
S = np.zeros(M)

In [47]:
for i in range(M):
    w = 0
    for j in range(N):
        w += (X[j,0]*X[j,0] + X[j,1]*X[j,1])*H(j,i,f1,fh,M,N,Fs)
    S[i] = np.log(w)
    

In [48]:
c = np.zeros(M)

In [49]:
for i in range(M):
    w = 0
    for j in range(M):
        w += S[j]*np.cos(np.pi*i*(j+1/2)/M)
    c[i] = w

In [50]:
#Мэл-коэффициенты
c

array([-1.16148416e+03, -7.12855977e+01, -5.04389650e+00, -1.40301450e+01,
       -7.61355756e+00, -8.05122442e+00, -2.53451927e+00, -1.22602209e+00,
       -7.26771688e-02, -1.20027571e+00,  4.31230301e-01, -1.18776155e+00,
        3.04451867e+00, -8.72898456e-01,  1.37632488e+00, -8.09636648e-01,
        1.08375702e+00, -1.48964324e-01, -5.36012151e-01, -9.76382921e-01,
       -3.08904012e+00,  2.42712392e-01,  7.51063850e-02,  1.33352166e+00,
        7.35957763e-01,  1.41974260e+00])